# TV Script Generation

Here we will try to generate _fake_ TV script for Seinfeld using a RNN based neural network.

In [ ]:
import os
from collections import Counter
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

## Utility functions

In [ ]:
SPECIAL_WORDS = {'PADDING': '<PAD>'}

def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data


def load_preprocess():
    """
    Load the Preprocessed Training data 
    """
    return pickle.load(open('preprocess.p', mode='rb'))


def save_model(filename, decoder):
    """
    Save the trained model
    """    
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)


def load_model(filename):
    """
    Load the trained model
    """    
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    return torch.load(save_filename)

## Load and Explore the Data

In [ ]:
## load the data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = load_data(data_dir)

In [2]:
## Explore the data
view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143
The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! yo

## Pre-process Data

In [ ]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punct = {'.': '||Period||',
             ',': '||Comma||',
             '"': '||Quotation_Mark||',
             ';': '||Semicolon||',
             '!': '||Exclamation_mark||',
             '?': '||Question_mark||',
             '(': '||Left_Parentheses||',
             ')': '||Right_Parentheses||',
             '-': '||Dash||',
             '\n': '||Return||'    
    }
    
    return punct

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Collect all unique words sorted by their count in desc order
    word_count = Counter(text)
    word_count = sorted(word_count, key=word_count.get, reverse=True)
    
    # Assign ids to each word starting from 1
    vocab_to_ints = {word:id for id, word in enumerate(word_count)}
    
    # Swap keys and values in vocab_to_ints to create ints_to_vocab dict
    ints_to_vocab = {v:k for k, v in vocab_to_ints.items()}
    # return tuple
    return (vocab_to_ints, ints_to_vocab)


def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocess Text Data
    """
    text = load_data(dataset_path)
    
    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))

In [5]:
# pre-process training data
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [6]:
# We can re-load the pre-processed data If we ever decide to come back to this notebook or have to restart the notebook.
int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

## Build the data batch

In [10]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # Compute number of full batches that we can create
    n_batches = len(words) // sequence_length
    
    # If data is not completely split between full batches there is spill over
    # then we need pad last sequence with leading <PAD> to make them of
    # sequence_length 
    padding = None
    if n_batches * sequence_length < len(words):
        pad_length = sequence_length - (len(words) - (n_batches * sequence_length))
        pad_char = SPECIAL_WORDS['PADDING']
        pad_char = vocab_to_int[pad_char]
        padding = [pad_char] * pad_length
    
    # Iterate over the words corpus n_batches times and create 
    # tuples of input-sequence tensor and corresponding target tensor.
    # zero pad last sequence, if required
    input_seq, target = [], []
    for i in range(n_batches):
        start_idx = i * sequence_length
        end_idx = start_idx + sequence_length
        # sequence_length long sequence of words
        _input = words[start_idx:end_idx]
        # try immediate next word after the sequence as target
        # however, if no next word available then assign very first word 
        # of the corpus as the target
        try:
            _target = words[end_idx]
        except(IndexError):
            _target = words[0]
        # store input-seq and target 
        input_seq.append(_input)
        target.append(_target)
    else:
        # If spill-over, then include very last seq with leading zero-padding
        if padding:
            # Get available last words as last sequence
            last_seq = list(words[end_idx:])
            # Pad with leading zeroes, target would be the very first 
            # word of the corpus
            last_seq[0:0] = padding
            _input = last_seq
            _target = words[0]
            input_seq.append(_input)
            target.append(_target)
    
    # Create dataset
    ds = TensorDataset(torch.tensor(input_seq).long(), torch.tensor(target).long())
    
    # Create dataloader
    dl =  DataLoader(ds, batch_size=batch_size)
    
    # return a dataloader
    return dl

dl = batch_data(words=int_text, sequence_length=5, batch_size=10)
batch = next(iter(dl))
print(batch[0].shape)
print(batch[0])
print(batch[1].shape)
print(batch[1])

torch.Size([10, 5])
tensor([[   7,   34,    4,   27,   18],
        [  24,   22,   50,   58,    3],
        [  34,    4,   27,    2,   83],
        [ 120,   62,    3,    8,   54],
        [  47,    2,   24,   22,   47],
        [   1,    1,    1,   17,   47],
        [  22,   82,   20,    6, 1252],
        [ 545, 8782, 7189,   20,  241],
        [   1,  149,    1,    1,    1],
        [  84,    4,  200,  238,  149]])
torch.Size([10])
tensor([ 24,  34, 120,  47,   1,  22, 545,   1,  84, 208])


In [12]:
# test the batch preparation
test_text = range(0, 53)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

# running in loop to check two batches to check spill-over and padding handling
for sample_x, sample_y in t_loader:
    print(sample_x.shape)
    print(sample_x)
    print()
    print(sample_y.shape)
    print(sample_y)
    print()

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49]])

torch.Size([10])
tensor([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

torch.Size([1, 5])
tensor([[21387, 21387,    50,    51,    52]])

torch.Size([1])
tensor([0])



## Build the Network

In [3]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # define model layers
        self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, 
                          hidden_size=hidden_dim, 
                          num_layers=n_layers, 
                          batch_first=True, 
                          dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        # nn_input shape = (batch_size, sequence_length, feature_dim) --> (64, 15, 1)
        # hidden shape = (n_layers, batch_size, hidden_dim) --> (2, 64, 512)
        
        # x embedding o/p shape = (batch_size, embedding_dim) --> (64, 300)
        x = self.embedding(nn_input)
        
        # x gru o/p shape = (batch_size, sequence_length, hidden_dim) --> (64, 15, 512)
        x, hidden = self.gru(x, hidden)
        x = self.dropout(x)
        
        # reshape GRU output such that 2nd dim = hidden_dim so that we can feed 
        # it to fc layer --> (64 * 15, 512)
        x = x.reshape(-1, self.hidden_dim)
        
        # output fc o/p shape = (batch_size*sequence_length, output_size) --> (64 * 15, vocab_size)
        output = self.fc(x)
        
        # reshpe fc output to make first dim = batch_size and last dim = output_size
        batch_size = nn_input.shape[0]
        sequence_length = nn_input.shape[1]
        output = output.reshape(batch_size, sequence_length, self.output_size)
        
        # extract all outputs for last time-step (-1 below) for all batches
        output = output[:, -1, :]
        
        # return one batch of output word scores and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # initialize hidden state with zeros and move it to GPU if available        
        if (train_on_gpu):
            hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda()
        else:
            hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        
        return hidden

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

In [ ]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    optimizer.zero_grad()
    
    # detach hidden state from computation graph
    hidden = hidden.data
    output, hidden = rnn(inp, hidden)    
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

In [15]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [16]:
# Data params
# Sequence Length
sequence_length = 15  # of words in a sequence
# Batch Size
batch_size = 64

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [19]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.0007 

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

In [ ]:
# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [20]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.568603610992431
Epoch:    2/10    Loss: 4.7756297567817185
Epoch:    3/10    Loss: 4.38765674001819
Epoch:    4/10    Loss: 4.071462697695352
Epoch:    5/10    Loss: 3.7681871848932764
Epoch:    6/10    Loss: 3.480992128825932
Epoch:    7/10    Loss: 3.196643516758158
Epoch:    8/10    Loss: 2.948043690559297
Epoch:    9/10    Loss: 2.709813594689795
Epoch:   10/10    Loss: 2.509170540952323
Model Trained and Saved


## Test the Network 
With the network trained and saved, we'll use it to generate a new, "fake" Seinfeld TV script

In [22]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [23]:
# You can choose a different length
gen_length = 400 
# You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script.
prime_word = 'elaine'
# Generate script and print
pad_word = SPECIAL_WORDS['PADDING']

# Load the model
_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
trained_rnn = load_model('trained_rnn')

generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

elaine: anyway?

kramer: well, i know, i don't think so, i know, i think i can have it to be right.

jerry:(looking at the met) oh, no, thank you.

[setting: indian apartment]

kruger: hi.

george:(trying to whisper) what are you doing?

george: well, i'm just not really much.

elaine: you know, i don't wanna know, i'm gonna be stuck in the mood.

jerry: you know, that was just that was one, that was all the way, then it!!!

jerry: just got it!

kruger: no! you're no! i mean, i was excited!

elaine:(laughing) hey, what are you doing here?

george: well, i think i think its the mood.

jerry: so you can go, so, how about your name?

george: i think i was in the mood!

jerry:(looking up the door) oh, that's right.

george: i think that. you know, i can't believe that i think i can't believe this, but if i can tell you that it's a little time. it's just a whole story.

kramer: yeah, i don't think that's so.

elaine: what are you doing here?

peterman: i got an extra.

jerry: well, i gotta 

In [24]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()